<a href="https://colab.research.google.com/github/bryanjjones/InSiTE/blob/master/Site_Integration_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#%%capture
#@title 1. Install

# check number of cores
import multiprocessing
cores = str(multiprocessing.cpu_count())
print("Number of cores: " + cores)

#@markdown Please execute each cell by pressing the *Play* button on
#@markdown the left.

# download github "https://github.com/bryanjjones/InSiTE" and install dependencies
!git clone https://github.com/bryanjjones/InSiTE.git
!pip install -r InSiTE/requirements.txt

# move to InSiTE directory
%cd InSiTE/

# install bowtie
!wget https://sourceforge.net/projects/bowtie-bio/files/bowtie2/2.5.1/bowtie2-2.5.1-linux-x86_64.zip -P bins/
!unzip bins/bowtie2-2.5.1-linux-x86_64.zip -d bins/
!mv bins/bowtie2-2.5.1-linux-x86_64 bins/bowtie2

# install TwoBitToFa
# look here: http://hgdownload.cse.ucsc.edu/admin/exe/linux.x86_64/
#!wget http://hgdownload.cse.ucsc.edu/admin/exe/linux.x86_64/twoBitToFa
#path_to_twobittofa = "twoBitToFa"

# install bbmap
!wget https://sourceforge.net/projects/bbmap/files/BBMap_39.03.tar.gz
!tar -xvzf BBMap_39.03.tar.gz
!mv bbmap bins/bbmap
#bbmerge_location = "bbmap/bbmerge-auto.sh"

# install bedtools
!wget https://github.com/arq5x/bedtools2/releases/download/v2.31.0/bedtools-2.31.0.tar.gz
!tar -xvzf bedtools-2.31.0.tar.gz
# add to path
%cd bedtools2
!make
%cd ..
import os
os.environ['PATH'] += ':/content/InSiTE/bedtools2/bin'

# download genome files
!mkdir reference_datasets/genomes
#!gsutil -m cp -r gs://genomics-public-data/references/GRCh38 reference_datasets/genomes/
!wget https://hgdownload.cse.ucsc.edu/goldenPath/hg38/bigZips/hg38.2bit -P reference_datasets/genomes/

# download reference index
!mkdir reference_datasets/genomes/GRCh38.fna.bowtie_index
!wget ftp.ncbi.nlm.nih.gov/genomes/archive/old_genbank/Eukaryotes/vertebrates_mammals/Homo_sapiens/GRCh38/seqs_for_alignment_pipelines/GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.bowtie_index.tar.gz
!tar -xvzf GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.bowtie_index.tar.gz -C reference_datasets/genomes/GRCh38.fna.bowtie_index/

# download annotations
!gdown --folder https://drive.google.com/drive/folders/1WCWZyEOAJxNd9g36fohQii_G7OVUDQ9c
!mv annotations reference_datasets/annotations

In [ ]:
#@title 2. Link Google Drive

#@markdown Running this code will create a popup asking to authorize Colab to access your Google Drive. <br>
#@markdown *Note that you'll have to re-authorize Colab access with every new runtime.*

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
#@title 2. Upload Vector file
vector_file = False #@param {type:"boolean"}
#@markdown This will eliminate reads that align to plasmid and not integrated transposon
if vector_file:
  from google.colab import files
  uploaded = files.upload()
  tmp = list(uploaded.keys())
  vector_file = tmp[0]

In [ ]:
#@title 3. Run analysis
import os
import re
#@markdown Select a folder to run all paired reads with the same conditions. Leave blank to run specific files.
folder = "" #@param{type:"string"}
fastq_file = "/content/gdrive/MyDrive/data/30-933683439/NGS1_R1_001.fastq.gz" #@param {type:"string"}
paired_fastq_file = "/content/gdrive/MyDrive/data/30-933683439/NGS1_R2_001.fastq.gz" #@param {type:"string"}

random_sites = False #@param {type:"boolean"}
random_nt = False #@param {type:"boolean"}
no_seqs = False #@param {type:"boolean"}
#compress_reads = True #@param {type:"boolean"}
no_annotate = False #@param {type:"boolean"}
barcode = "NNNNNN"  #@param {type:"string"}
lwindow = 0 #@param {type:"integer"}
rwindow = 0 #@param {type:"integer"}
min = 25 #@param {type:"integer"}
primer5 = "" #@param {type:"string"}
primer3 = ""  #@param {type:"string"}
trim5 = 0 #@param {type:"integer"}
trim3 = 0 #@param {type:"integer"}
feature = "None" #@param ["None", "gene", "exon", "intron"]
dist = False #@param {type:"boolean"}
close = 100 #@param {type:"integer"}

paired_files = {}
if folder:
  results_dir = folder
  for file in os.listdir(folder):
    if '.fastq' in file and 'merged' not in file and 'trimmed' not in file:
      try:
        paired_files[re.sub(r'_R\d_', '', file)].append(os.path.join(folder, file))
      except KeyError:
        paired_files[re.sub(r'_R\d_', '', file)] = [os.path.join(folder, file)]
else:
  results_dir = os.path.dirname(fastq_file)
  paired_files['OnlyOne'] = [fastq_file, paired_fastq_file]

#run analysis
for key, value in paired_files.items():
  assert len(value) < 3, f"Too many files linked to '{value[0]}'"
  fastq_file = value[0]
  paired_fastq_file = value[1]
  command = "python scripts/InSiTE.py --fastq=" + fastq_file
  if paired_fastq_file:
    command += " --pairs=" + paired_fastq_file
  #command += " --chromosome_ids=reference_datasets/chromosomes.csv"
  #command += " --bowtieindex=reference_datasets/genomes/GRCh38.fna.bowtie_index/GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.bowtie_index"
  #command += " --twobitlocation=scripts/TwoBitToFa"
  command += " --twobitgenomelocation=reference_datasets/genomes/hg38.2bit"
  command += " --bowtielocation=bins/bowtie2/bowtie2"
  command += " --bowtiethreads=" + cores
  #command += " --bbmergelocation=$bbmerge_location"
  #command += " --bbduklocation=bbmap/bbduk.sh"
  if vector_file:
    command += " --vectors=" + vector_file
  if random_sites:
    command += " --random_is"
  if random_nt:
    command += " --random_nt"
  if no_seqs:
    command += " --no_seqs"
  #if compress_reads:
  #  command += " -z"
  if no_annotate:
    command += " --no_annotate"
  if barcode != "NNNNNN":
    command += " --barcode=" + barcode
  if lwindow != 0:
    command += " --lwindow=" + str(lwindow)
  if rwindow != 0:
    command += " --rwindow=" + str(rwindow)
  if min != 25:
    command += " --min=" + str(min)
  if primer5:
    command += " --primer5=" + primer5
  if primer3:
    command += " --primer3=" + primer3
  if trim5 != 0:
    command += " --trim5=" + str(trim5)
  if trim3 != 0:
    command += " --trim3=" + str(trim3)
  if feature != "None":
    command += " --feature=" + feature
  if dist:
    command += " --dist"
  if close != 100:
    command += " --close=" + str(close)

  # run analysis
  print(command)
  !$command
  !rm $results_dir/*trimmed.fastq
  !rm $results_dir/*merged_reads.fastq
  !rm $results_dir/*.bam


python scripts/InSiTE.py --fastq=/content/gdrive/MyDrive/data/30-933683439/NGS1_R2_001.fastq.gz --pairs=/content/gdrive/MyDrive/data/30-933683439/NGS1_R1_001.fastq.gz --twobitgenomelocation=reference_datasets/genomes/hg38.2bit --bowtielocation=bins/bowtie2/bowtie2 --bowtiethreads=8
Merging paired reads files /content/gdrive/MyDrive/data/30-933683439/NGS1_R2_001.fastq.gz and /content/gdrive/MyDrive/data/30-933683439/NGS1_R1_001.fastq.gz using bbmerge.
./bins/bbmap/bbmerge-auto.sh in1=/content/gdrive/MyDrive/data/30-933683439/NGS1_R2_001.fastq.gz in2=/content/gdrive/MyDrive/data/30-933683439/NGS1_R1_001.fastq.gz out=/content/gdrive/MyDrive/data/30-933683439/NGS1_R2_001_merge.fastq outu1=/content/gdrive/MyDrive/data/30-933683439/NGS1_R2_001_unmerge.fastq outu2=/content/gdrive/MyDrive/data/30-933683439/NGS1_R1_001_unmerge.fastq


In [ ]:
#@title Download Results
#get dir of fastq_file
results_dir = os.path.dirname(fastq_file)
print(results_dir)
# download results
%cd $results_dir
!rm results.zip
from google.colab import files
!zip results.zip *.csv *.svg
files.download("results.zip")
print('Also look in your google drive folder')